In [ ]:
# --- Widgets & layout (left controls + right plot area) ---
import ipywidgets as W
from ipywidgets import AppLayout, Layout
from IPython.display import display

# Controls
layers = sorted(meta["layer"].dropna().unique().tolist())
osi_min, osi_max  = float(meta["OSI"].min()), float(meta["OSI"].max())
sel_min, sel_max  = float(meta["selectivity"].min()), float(meta["selectivity"].max())

layer_w = W.SelectMultiple(
    options=layers, value=tuple(layers), description="Layer",
    layout=Layout(width="260px", height="120px")
)
osi_w   = W.FloatRangeSlider(
    description="OSI range", min=0, max=1, step=0.01,
    value=[max(0, osi_min), min(1, osi_max)], readout_format=".2f",
    layout=Layout(width="260px")
)
sel_w   = W.FloatRangeSlider(
    description="Sel range", min=0, max=1.2, step=0.01,
    value=[max(0, sel_min), min(1.2, sel_max)], readout_format=".2f",
    layout=Layout(width="260px")
)
plot_w  = W.ToggleButtons(
    options=["Tuning", "PSTH"], value="Tuning", description="Plot",
    layout=Layout(width="260px")
)
idx_w   = W.IntSlider(
    description="Unit index", min=0, max=len(meta)-1, step=1, value=0,
    continuous_update=False, layout=Layout(width="260px")
)

controls = W.VBox([layer_w, osi_w, sel_w, plot_w, idx_w],
                  layout=Layout(width="280px", min_width="280px", flex="0 0 280px"))

# Plot/output area — fixed min height so it can't overlap controls
plot_out = W.Output(layout=Layout(width="100%", min_height="640px",
                                  overflow="auto", border="1px solid #eee", padding="6px"))

# Optional footer for status
status_out = W.HTML(layout=Layout(margin="6px 0 0 0"))

# App-style layout: left sidebar = controls, center = plot
app = AppLayout(
    header=None,
    left_sidebar=controls,
    center=plot_out,
    right_sidebar=None,
    footer=status_out,
    pane_widths=["300px", "1fr", 0],
    pane_heights=[0, "auto", "40px"]
)
display(app)


In [ ]:
# --- Render logic ---
import matplotlib.pyplot as plt
import numpy as np

def filtered_indices():
    df = meta.copy()
    if layer_w.value:
        df = df[df["layer"].isin(list(layer_w.value))]
    lo, hi = osi_w.value
    df = df[(df["OSI"] >= lo) & (df["OSI"] <= hi)]
    lo2, hi2 = sel_w.value
    df = df[(df["selectivity"] >= lo2) & (df["selectivity"] <= hi2)]
    return df.index.to_numpy(), df

def render(*_):
    idxs, df = filtered_indices()
    n = len(df)

    with plot_out:
        plot_out.clear_output(wait=True)
        if n == 0:
            display(W.HTML("<b>No units match the filters.</b>"))
            status_out.value = "No matching units."
            return

        # Map slider to filtered set
        pos = min(idx_w.value, n-1)
        unit_global_idx = int(idxs[pos])
        row = meta.iloc[unit_global_idx]

        # Clean figure creation (prevents overlap)
        plt.close("all")
        fig = plt.figure(figsize=(8.5, 5.0), dpi=110)

        if plot_w.value == "Tuning":
            plt.plot(angles, tun_curves[unit_global_idx], marker="o")
            plt.title(f"Tuning — unit {int(row['unit_id'])} | layer={row['layer']} | OSI={row['OSI']:.2f}")
            plt.xlabel("Orientation (deg)"); plt.ylabel("Response (a.u.)")
        else:
            plt.plot(time, psth_curves[unit_global_idx])
            plt.title(f"PSTH — unit {int(row['unit_id'])} | layer={row['layer']} | OSI={row['OSI']:.2f}")
            plt.xlabel("Time (s)"); plt.ylabel("Rate (Hz)")

        plt.grid(True, alpha=0.25)
        plt.tight_layout()
        plt.show()

    status_out.value = f"<span>{n} units match filters. Showing {pos+1}/{n} (global idx {unit_global_idx}).</span>"

# Wire events
for w in (layer_w, osi_w, sel_w, plot_w, idx_w):
    w.observe(render, names="value")

render()


In [ ]:
# --- Render logic ---
def filtered_indices():
    df = meta.copy()
    if layer_w.value:
        df = df[df["layer"].isin(list(layer_w.value))]
    lo, hi = osi_w.value
    df = df[(df["OSI"] >= lo) & (df["OSI"] <= hi)]
    lo2, hi2 = sel_w.value
    df = df[(df["selectivity"] >= lo2) & (df["selectivity"] <= hi2)]
    return df.index.to_numpy(), df

def render(*_):
    idxs, df = filtered_indices()
    n = len(df)
    with plot_out:
        plot_out.clear_output(wait=True)
        if n == 0:
            display(W.HTML("<b>No units match the filters.</b>"))
            return
        # Map slider to filtered set
        pos = min(idx_w.value, n-1)
        unit_global_idx = idxs[pos]
        unit_row = meta.iloc[unit_global_idx]
        # Plot
        if plot_w.value == "Tuning":
            newfig()
            plt.plot(angles, tun_curves[unit_global_idx], marker="o")
            plt.title(f"Tuning — unit {int(unit_row['unit_id'])} | layer={unit_row['layer']} | OSI={unit_row['OSI']:.2f}")
            plt.xlabel("Orientation (deg)"); plt.ylabel("Response (a.u.)")
            plt.tight_layout(); plt.show()
        else:
            newfig()
            plt.plot(time, psth_curves[unit_global_idx])
            plt.title(f"PSTH — unit {int(unit_row['unit_id'])} | layer={unit_row['layer']} | OSI={unit_row['OSI']:.2f}")
            plt.xlabel("Time (s)"); plt.ylabel("Rate (Hz)")
            plt.tight_layout(); plt.show()
        # Info
        info_html = f"<b>{n}</b> units match filters. Showing {pos+1}/{n} (global index {unit_global_idx})."
        display(W.HTML(info_html))

# Wire events
for w in (layer_w, osi_w, sel_w, plot_w, idx_w):
    w.observe(render, names="value")
render()
